<a href="https://colab.research.google.com/github/winitkale/swara-transformer-hindustani/blob/main/Swara_Transformer_Hindustani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎶 Swara Transformer with Raga Selection & Live Monitoring
This notebook supports:
- Raga-based swara generation
- Alaap / Pakad styles
- Live microphone pitch capture

In [11]:
!pip install -q torchaudio transformers datasets matplotlib librosa ipywidgets sounddevice

In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random

raga_dropdown = widgets.Dropdown(
    options=['Yaman', 'Bhairav', 'Bhairavi', 'Kafi', 'Kalyan', 'Marwa', 'Poorvi', 'Asavari', 'Bilawal', 'Todi'],
    description='🎼 Raga:',
    style={'description_width': 'initial'}
)

style_dropdown = widgets.ToggleButtons(
    options=['Alaap', 'Pakad'],
    description='🎵 Style:',
    style={'description_width': 'initial'}
)

output_box = widgets.Output()

def generate_swaras(raga, style):
    alaap_patterns = ['S R G M', 'R G M P', 'M P D N', "N S' R' G'"]
    pakad_dict = {
        'Yaman': "N R G M D N S'",
        'Bhairav': "S r G M P d N S'",
        'Bhairavi': "S R g M P d n S'",
        'Kafi': "S R g M P D n S'",
        'Kalyan': "S R G M# P D N S'",
        'Marwa': "S r G M D N S'",
        'Poorvi': "S r G M# P d N S'",
        'Asavari': "S R g M P d n S'",
        'Bilawal': "S R G M P D N S'",
        'Todi': "S r g M# D n S'"
    }
    if style == 'Alaap':
        return '  |  '.join(random.choices(alaap_patterns, k=4))
    else:
        return pakad_dict.get(raga, "S R G M P D N S'")

def on_generate_clicked(b):
    with output_box:
        clear_output()
        raga = raga_dropdown.value
        style = style_dropdown.value
        print(f'🎼 Raga: {raga} | 🎵 Style: {style}')
        print('Generated Swaras:')
        print(generate_swaras(raga, style))

generate_button = widgets.Button(description='Generate Swaras')
generate_button.on_click(on_generate_clicked)

display(raga_dropdown, style_dropdown, generate_button, output_box)

Dropdown(description='🎼 Raga:', options=('Yaman', 'Bhairav', 'Bhairavi', 'Kafi', 'Kalyan', 'Marwa', 'Poorvi', …

ToggleButtons(description='🎵 Style:', options=('Alaap', 'Pakad'), style=ToggleButtonsStyle(description_width='…

Button(description='Generate Swaras', style=ButtonStyle())

Output()

## 🎙️ Live Mic Pitch Monitoring (Experimental)

In [ ]:
import sounddevice as sd
import numpy as np
import librosa
import matplotlib.pyplot as plt

DURATION = 5  # seconds
SAMPLE_RATE = 22050

def record_audio():
    print("🎙️ Recording... Sing now")
    audio = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1)
    sd.wait()
    return audio.flatten()

def plot_pitch(audio):
    pitches, magnitudes = librosa.piptrack(y=audio, sr=SAMPLE_RATE)
    pitch = []
    for i in range(pitches.shape[1]):
        index = magnitudes[:, i].argmax()
        pitch_val = pitches[index, i]
        if pitch_val > 0:
            pitch.append(pitch_val)
    times = np.linspace(0, DURATION, len(pitch))
    plt.figure(figsize=(10, 4))
    plt.plot(times, pitch)
    plt.title("🎙️ Detected Pitch Over Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.grid(True)
    plt.show()

# To use:
# audio = record_audio()
# plot_pitch(audio)


## ✅ Instructions:
- Use the dropdown to select raga and style.
- Click `Generate Swaras` to see patterns.
- Run the last two code cells to record and visualize pitch.